In [36]:
import pandas as pd
import datetime as dt
import os

In [4]:
def does_fuzzy_report_exist(pub_code):
    fuzzy_report_path = 'econlit_scopus_matching_out/{}_fuzzy_results/{}_fuzzy_matching_report.txt'.format(pub_code, pub_code)
    fuzzy_report_status = os.path.exists(fuzzy_report_path)

    return fuzzy_report_status


In [48]:
def get_report_timestamp(pub_code, report_type: str):
    if report_type == 'fuzzy':
        report_path = 'econlit_scopus_matching_out/{}_fuzzy_results/{}_fuzzy_matching_report.txt'.format(pub_code, pub_code)
    elif report_type == 'hand_selection':
        report_path = 'econlit_scopus_matching_out/{}_fuzzy_results/{}_hand_selection_report.txt'.format(pub_code, pub_code)

    
    
    with open(report_path, 'r') as report:
        report_lines = report.readlines()
        timestamp_line = report_lines[0]

        timestamp = dt.datetime.strptime(timestamp_line[:-1],'%Y-%m-%d %X.%f')
        return timestamp

In [46]:
def does_hand_selection_report_exist(pub_code):
    hand_selection_report_path = 'econlit_scopus_matching_out/{}_fuzzy_results/{}_hand_selection_report.txt'.format(pub_code, pub_code)

    hand_selection_report_status = os.path.exists(hand_selection_report_path)
    return hand_selection_report_status

In [40]:
def generate_hand_selection_report(pub_code):
    fuzzy_report_path = fuzzy_report_path = 'econlit_scopus_matching_out/{}_fuzzy_results/{}_fuzzy_matching_report.txt'.format(pub_code, pub_code)
    hand_selection_report_path = hand_selection_report_path = 'econlit_scopus_matching_out/{}_fuzzy_results/{}_hand_selection_report.txt'.format(pub_code, pub_code)

    fuzzy_report = open(fuzzy_report_path, 'r')
    fuzzy_report_lines = fuzzy_report.readlines()
    fuzzy_report.close()

    hand_selection_report_lines = fuzzy_report_lines

    current_timestamp = str(dt.datetime.now())+'\n'    
    report_title = 'HAND SELECTION_REPORT\n'

    hand_selection_report_lines[0] = current_timestamp
    hand_selection_report_lines[1] = report_title


    with open(hand_selection_report_path, 'w') as hand_selection_report:
        hand_selection_report.writelines(hand_selection_report_lines)

    return


In [54]:
if does_fuzzy_report_exist('RES'):
    if does_hand_selection_report_exist('RES'):
        print('haha')
    else:
        print("It looks like a hand-selection report for _______ doesn't exist yet. Generating one now.")
        generate_hand_selection_report('RES')
else:
    print("It looks like a fuzzy matching report for _______doesn't exist yet. Please first run the fuzzy-matching algorithm")


fuzzy_report_creation_time = get_report_timestamp('RES', 'fuzzy')
hand_selection_report_time = get_report_timestamp('RES', 'hand_selection')


print(fuzzy_report_creation_time)

print(hand_selection_report_time)
fuzzy_report_creation_time < dt.datetime.now()


haha
2022-07-06 22:38:24.964131
2022-07-06 22:38:35.186346


True